## Single-armed cluster tool with concurrent flow wafers 

* The stage configuration is [1,1,1,1,1,1]
    * Type 1 wafers are processed in the order PM1 -> PM2 -> PM3. 
    * Type 2 wafers are processed in the order PM4 -> PM5 -> PM6. 
    
* Process times are sampled from values between 10-300s. 
* The baseline robot move sequence uses concurrent backward sequence. 
* Trained model checkpoint file is loaded from ./saved_models/checkpoint_v4.pt
    * Another trained model for (Type 1 flow is PM1->PM2, Type 2 flow is PM3->PM4) is ./saved_models/checkpoint_v1.pt

In [ ]:
import copy
import logging
import argparse
import torch
import random
import numpy as np
import pandas as pd

from envs.sdcfEnv import sdcfEnv as Env, State
from model.model_concat import CONCATNet as CONCATModel
from envs.algorithms.cbs import ConcurrentBackwardSequence

# Global configurations
DEBUG_MODE = True
USE_CUDA = not DEBUG_MODE
CUDA_DEVICE_NUM = 0
SEED = 1000

def set_seed(seed=SEED):
    """Fix random seed for reproducibility."""
    torch.backends.cudnn.deterministic = True
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

def parse_arguments():
    """Parse command line arguments."""
    parser = argparse.ArgumentParser(description='Train a model with a specific number of lot types')
    parser.add_argument('--foup_size', type=int, default=100, help='Size of the foup')
    parser.add_argument('--group1_stage', type=int, default=1, help='Stages for type 1')
    parser.add_argument('--group1_min_prs_time', type=int, default=10, help='Minimum processing time for type 1')
    parser.add_argument('--group1_max_prs_time', type=int, default=300, help='Maximum processing time for type 1')
    parser.add_argument('--group2_stage', type=int, default=1, help='Stages for type 2')
    parser.add_argument('--group2_min_prs_time', type=int, default=10, help='Minimum processing time for type 2')
    parser.add_argument('--group2_max_prs_time', type=int, default=300, help='Maximum processing time for type 2')
    parser.add_argument('--prod_quantity', type=int, default=10, help='Production quantity (Unit: FOUP)')
    parser.add_argument('--done_quantity', type=int, default=100, help='Done Production quantity (Unit: Wafer)')
    parser.add_argument('--num_lot_type', type=int, default=2, help='Total number of lot types')

    parser.add_argument('--model_type', type=str, default='concat', help='Model type = {is, rms, concat}')
    parser.add_argument('--input_action', type=str, default='wafer', help='loadlock input action type = {wafer, type}')

    return parser.parse_args([])


def get_stage_list():
    """Define stage configurations."""
    return [
        [1,1],
        [1,1,1],
    ]

def setup_tester_params(args):
    """Initialize tester parameters."""
    return {
        'use_cuda': False,
        'cuda_device_num': CUDA_DEVICE_NUM,
        'model_load': {
            'enable': True,
            'path': f'./saved_models/',
        },
        'multi_run_size': 1,
        'problem_count': 100,
        'test_batch_size': 100,
    }
    
def setup_env_params(args, stage_list):
    """Initialize environment parameters."""
    return {
        'foup_size': args.foup_size,
        'group1_stage': stage_list[args.group1_stage],
        'group1_min_prs_time': args.group1_min_prs_time,
        'group1_max_prs_time': args.group1_max_prs_time,
        'group2_stage': stage_list[args.group2_stage],
        'group2_min_prs_time': args.group2_min_prs_time,
        'group2_max_prs_time': args.group2_max_prs_time,
        'prod_quantity': args.prod_quantity,
        'done_quantity': args.done_quantity,
        'num_lot_type': args.num_lot_type,
    }
    
def setup_model_params(args, env_params):
    """Initialize model parameters."""
    return {
        'type': args.model_type,
        'input_action': args.input_action,
        'purge': False,
        'embedding_dim': 256,
        'sqrt_embedding_dim': 256**(1/2),
        'encoder_layer_num': 3,
        'qkv_dim': 16,
        'sqrt_qkv_dim': 16**(1/2),
        'head_num': 16,
        'logit_clipping': 10,
        'ff_hidden_dim': 512,
        'ms_hidden_dim': 16,
        'ms_layer1_init': (1/2)**(1/2),
        'ms_layer2_init': (1/16)**(1/2),
        'eval_type': 'argmax',
        'normalize': 'instance'
    }


def main(): 
    set_seed() 
    args = parse_arguments()
    stage_list = get_stage_list()
    
    env_params = setup_env_params(args, stage_list)
    model_params = setup_model_params(args, env_params)
    tester_params = setup_tester_params(args)
    
    tester = Tester(env_params=env_params,
                      model_params=model_params,
                      tester_params=tester_params)
    
    result = tester.run()    
    return result


class Tester:
    def __init__(self, env_params, model_params, tester_params):
        self.env_params = env_params
        self.model_params = model_params
        self.tester_params = tester_params
        self.device = torch.device('cuda' if tester_params['use_cuda'] else 'cpu')
        self.model_params['device'] = self.device
        self.model = self._load_model()
        
    def _load_model(self):
        model = CONCATModel(**self.env_params, **self.model_params)
        model_load = self.tester_params['model_load']
        if model_load['enable']:
            checkpoint_path = f"{model_load['path']}/checkpoint_v4.pt"
            checkpoint = torch.load(checkpoint_path, map_location=self.device)
            model.load_state_dict(checkpoint['model_state_dict'])
            print(f'[Saved Model Loaded...] -> {checkpoint_path}')
        return model

    def run(self):
        """Execute the testing process."""  
        print("Running tester...")
        # Implement the testing process here
        def _stack_states(states: list):
            return State(**{field: torch.stack([getattr(state, field) for state in states])
                            for field in State.__dataclass_fields__})

        # call environments & copy for each policy
        envs = []
        states = []
        for _ in range(self.tester_params['test_batch_size']):
            env = Env(**self.env_params)
            state = env.reset()
            envs.append(env)
            states.append(state)
            
        envs_ceil = copy.deepcopy(envs)
        envs_floor = copy.deepcopy(envs)
        envs_rl = copy.deepcopy(envs)
        
        # results storage
        results = []
        
        # run css policy
        ceil_makespans = []
        for _, e in enumerate(envs_ceil):
            policy_ceil = ConcurrentBackwardSequence(env, strategy='ceil')
            while not e.done:
                action = policy_ceil(e)
                _ = e.step(action)
            ceil_makespans.append(e.clock)
    
        floor_makespans = []
        for _, e in enumerate(envs_floor):
            policy_floor = ConcurrentBackwardSequence(env, strategy='floor')
            while not e.done:
                action = policy_floor(e)
                _ = e.step(action)
            floor_makespans.append(e.clock)

        cbs_makespans = [ceil_makespans[i] if ceil_makespans[i] < floor_makespans[i] 
                         else floor_makespans[i] for i in range(len(ceil_makespans))]
        
        # prepare rl state
        state = _stack_states(states)
        state.batch_idx = torch.arange(state.batch_size())
        state.to(self.device)
        
        self.model.eval()
        self.model.to(self.device)
        self.model.encoding(state)
        policy_rl = self.model
        rl_makespans = [1e10 for _ in range(state.batch_size())]
        
        while not state.done.all():
            action, prob = policy_rl(state)
            states = []
            for b, a in enumerate(action):
                state = envs_rl[b].step(a.item())
                states.append(state)
                if envs_rl[b].done and rl_makespans[b] == 1e10:
                    rl_makespans[b] = copy.deepcopy(envs_rl[b].clock)
                    ## env.done之后clock也还是在继续走，所以只记录第一次到达done的makespan
                    ## 用rl_makespans==1e10来判断是否是第一次到达done
            state = _stack_states(states)
            state.batch_idx = torch.arange(state.batch_size())
            state.to(self.device)
            
        # collect results
        for instance_id in range(self.tester_params['test_batch_size']):
            type1_time = [int(i) for i in envs[instance_id].recipes[0].time[1:-1]]
            type2_time = [int(i) for i in envs[instance_id].recipes[1].time[1:-1]]
            results.append([instance_id, type1_time, type2_time, int(cbs_makespans[instance_id]), int(rl_makespans[instance_id])])
        
        # convert to DataFrame and print
        df = pd.DataFrame(results, columns=["InstanceID", "Type 1 Process Time", "Type 2 Process Time", "CBS Makespan", "RL Makespan"])
        return df

In [ ]:
df = main()

# print average results
print("="*60)
print("Dual-armed cluster tool with concurrent flow wafers")
print(f'Average makespan of CBS: {df["CBS Makespan"].mean():.2f}, RL: {df["RL Makespan"].mean():.2f}')
print("="*60)

In [ ]:
# show dataframe tables 
pd.set_option('display.max_rows', None)  
pd.set_option('display.max_columns', None) 
pd.set_option('display.width', None)  
pd.set_option('display.max_colwidth', None)  
df